In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("vectorbt_reports/fixed_master_backtest_stats.csv")
display(df)

,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],...,Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio,asset
0,2024-06-03,2025-04-01,208 days,10000.0,9941.408340,-0.585917,15.028606,100.0,418.405555,15.146652,...,-2.566236,10 days 14:24:00,2 days 02:24:00,0.978639,-2.929583,0.064705,-0.067731,1.011706,0.084075,AAPL
1,2024-06-03 00:00:00,2025-04-01 00:00:00,208 days 00:00:00,10000.0,9031.667793,-9.683322,7.754851,100.0,370.714834,16.915044,...,-2.574415,11 days 17:08:34.285714285,3 days 00:00:00,0.695521,-48.416610,-0.502342,-0.967559,0.912633,-0.655865,AMZN
2,2024-06-01 00:00:00,2025-04-01 00:00:00,305 days 00:00:00,10000.0,8451.678168,-15.483218,25.790909,100.0,515.698917,26.322412,...,-3.162296,9 days 12:00:00,2 days 06:40:00,0.727364,-59.550840,-0.499524,-0.692730,0.894407,-0.703351,BTC-USD
3,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9670.290965,-3.297090,22.523686,100.0,357.641793,26.202416,...,-3.016455,7 days 19:12:00,4 days 20:18:27.692307692,0.914804,-18.317169,0.105999,-0.217042,1.021927,0.156094,BTC=F
4,2024-06-02 00:00:00,2025-03-31 00:00:00,216 days 00:00:00,10000.0,9624.789590,-3.752104,1.980042,100.0,417.453416,8.777077,...,-0.540836,12 days 04:48:00,3 days 00:00:00,0.564939,-17.867162,-0.872353,-0.712992,0.853281,-1.209504,CHFUSD=X
5,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,8148.995954,-18.510040,-4.068990,100.0,360.205747,29.376987,...,-3.204745,7 days 15:00:00,4 days 05:32:18.461538461,0.498720,-88.143050,-1.082658,-1.023110,0.826618,-1.442928,CL=F
6,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9462.618043,-5.373820,0.115232,100.0,349.428215,5.615763,...,-0.826009,15 days 06:00:00,3 days 17:08:34.285714285,0.523786,-29.854553,-1.546911,-1.637500,0.762782,-1.900394,DX-Y_NYB
7,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9497.316976,-5.026830,-0.123951,100.0,369.554491,5.645762,...,-0.771137,16 days 00:00:00,2 days 19:12:00,0.555075,-26.457001,-1.424905,-1.525659,0.774110,-1.754573,DX=F
8,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,9966.207765,-0.337922,7.121620,100.0,409.937573,7.435217,...,-0.947395,11 days 21:00:00,2 days 00:00:00,0.970995,-1.689612,-0.003304,-0.079272,0.999425,-0.004495,ES=F
9,2024-06-03 00:00:00,2025-04-01 00:00:00,209 days 00:00:00,10000.0,11217.513941,12.175139,32.911437,100.0,438.773832,6.565787,...,-0.982984,10 days 14:24:00,1 days 19:38:10.909090909,2.099896,57.976854,1.451683,3.384113,1.278276,1.991633,GC=F


In [8]:
sharpe_column = df["Sharpe Ratio"]
print(sharpe_column.mean())

-0.4227030385523384
